In [13]:
import json

In [14]:
import findspark
findspark.init()

In [15]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import split, explode
spark = SparkSession.builder.appName('read JSON files').getOrCreate()

In [16]:
# json_df=spark.read.option("inferSchema","true") \
#                 .option("header","true") \
#                 .option("sep",",") \
#                 .json("Mar*.json")

In [17]:
json_df=spark.read.json("tweets.txt")

In [18]:
json_df=spark.read.json("Apr_tweets*.json")

In [19]:
# number of files
json_df.count()

97

In [20]:
# show the schema
json_df.printSchema()

root
 |-- data: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- author_id: string (nullable = true)
 |    |    |-- created_at: string (nullable = true)
 |    |    |-- geo: struct (nullable = true)
 |    |    |    |-- coordinates: struct (nullable = true)
 |    |    |    |    |-- coordinates: array (nullable = true)
 |    |    |    |    |    |-- element: double (containsNull = true)
 |    |    |    |    |-- type: string (nullable = true)
 |    |    |    |-- place_id: string (nullable = true)
 |    |    |-- id: string (nullable = true)
 |    |    |-- public_metrics: struct (nullable = true)
 |    |    |    |-- like_count: long (nullable = true)
 |    |    |    |-- quote_count: long (nullable = true)
 |    |    |    |-- reply_count: long (nullable = true)
 |    |    |    |-- retweet_count: long (nullable = true)
 |    |    |-- text: string (nullable = true)
 |-- includes: struct (nullable = true)
 |    |-- places: array (nullable = true)
 |    |    |

In [21]:
# show the schema for tweets
json_df.select('data').printSchema()

root
 |-- data: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- author_id: string (nullable = true)
 |    |    |-- created_at: string (nullable = true)
 |    |    |-- geo: struct (nullable = true)
 |    |    |    |-- coordinates: struct (nullable = true)
 |    |    |    |    |-- coordinates: array (nullable = true)
 |    |    |    |    |    |-- element: double (containsNull = true)
 |    |    |    |    |-- type: string (nullable = true)
 |    |    |    |-- place_id: string (nullable = true)
 |    |    |-- id: string (nullable = true)
 |    |    |-- public_metrics: struct (nullable = true)
 |    |    |    |-- like_count: long (nullable = true)
 |    |    |    |-- quote_count: long (nullable = true)
 |    |    |    |-- reply_count: long (nullable = true)
 |    |    |    |-- retweet_count: long (nullable = true)
 |    |    |-- text: string (nullable = true)



In [22]:
# convert array to dict
data_df=json_df.select('data').withColumn('data', explode('data').alias('data'))

In [23]:
data_df.printSchema()

root
 |-- data: struct (nullable = true)
 |    |-- author_id: string (nullable = true)
 |    |-- created_at: string (nullable = true)
 |    |-- geo: struct (nullable = true)
 |    |    |-- coordinates: struct (nullable = true)
 |    |    |    |-- coordinates: array (nullable = true)
 |    |    |    |    |-- element: double (containsNull = true)
 |    |    |    |-- type: string (nullable = true)
 |    |    |-- place_id: string (nullable = true)
 |    |-- id: string (nullable = true)
 |    |-- public_metrics: struct (nullable = true)
 |    |    |-- like_count: long (nullable = true)
 |    |    |-- quote_count: long (nullable = true)
 |    |    |-- reply_count: long (nullable = true)
 |    |    |-- retweet_count: long (nullable = true)
 |    |-- text: string (nullable = true)



In [24]:
# number of records
data_df.count()

47978

In [25]:
data_df=data_df.select('data.author_id',
                       'data.created_at',
                       'data.geo.place_id',
                       'data.id',
                       'data.public_metrics',
                       'data.text')

In [34]:
data_df.printSchema()

root
 |-- author_id: string (nullable = true)
 |-- created_at: string (nullable = true)
 |-- place_id: string (nullable = true)
 |-- id: string (nullable = true)
 |-- public_metrics: struct (nullable = true)
 |    |-- like_count: long (nullable = true)
 |    |-- quote_count: long (nullable = true)
 |    |-- reply_count: long (nullable = true)
 |    |-- retweet_count: long (nullable = true)
 |-- text: string (nullable = true)



In [27]:
# select the place content
place_df=json_df.select('includes.places')

In [28]:
# show the schema fo place
place_df.printSchema()

root
 |-- places: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- country: string (nullable = true)
 |    |    |-- country_code: string (nullable = true)
 |    |    |-- full_name: string (nullable = true)
 |    |    |-- id: string (nullable = true)



In [29]:
# select the user content
user_df=json_df.select('includes.users')

In [30]:
# show the schema fo user
user_df.printSchema()

root
 |-- users: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- id: string (nullable = true)
 |    |    |-- location: string (nullable = true)
 |    |    |-- name: string (nullable = true)
 |    |    |-- username: string (nullable = true)



In [35]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [36]:
def read_nested_json(df):
    column_list = []

    for column_name in df.schema.names:
        print("Outside isinstance loop: " + column_name)
        # Checking column type is ArrayType
        if isinstance(df.schema[column_name].dataType, ArrayType):
            print("Inside isinstance loop of ArrayType: " + column_name)
            df = df.withColumn(column_name, explode(column_name).alias(column_name))
            column_list.append(column_name)

        elif isinstance(df.schema[column_name].dataType, StructType):
            print("Inside isinstance loop of StructType: " + column_name)
            for field in df.schema[column_name].dataType.fields:
                column_list.append(col(column_name + "." + field.name).alias(column_name + "_" + field.name))
        else:
            column_list.append(column_name)

    # Selecting columns using column_list from dataframe: df
    df = df.select(column_list)
    return df

In [37]:
def flatten_nested_json(df):
    read_nested_json_flag = True
    while read_nested_json_flag:
        print("Reading Nested JSON File ... ")
        df = read_nested_json(df)
        df.show(100, False)
        read_nested_json_flag = False

        for column_name in df.schema.names:
            if isinstance(df.schema[column_name].dataType, ArrayType):
              read_nested_json_flag = True
            elif isinstance(df.schema[column_name].dataType, StructType):
              read_nested_json_flag = True
    return df

In [38]:
data_df=flatten_nested_json(data_df)
data_df.show(500, False)

Reading Nested JSON File ... 
Outside isinstance loop: author_id
Outside isinstance loop: created_at
Outside isinstance loop: place_id
Outside isinstance loop: id
Outside isinstance loop: public_metrics
Inside isinstance loop of StructType: public_metrics
Outside isinstance loop: text
+-------------------+------------------------+----------------+-------------------+-------------------------+--------------------------+--------------------------+----------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|author_id          |created_at              |place_id        |id                 |public_metrics_like_count|public_metrics_quote_count|public_metrics_reply_count|public_metrics_retweet_count|text                

In [39]:
data_df.show(10, False)

+-------------------+------------------------+----------------+-------------------+-------------------------+--------------------------+--------------------------+----------------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|author_id          |created_at              |place_id        |id                 |public_metrics_like_count|public_metrics_quote_count|public_metrics_reply_count|public_metrics_retweet_count|text                                                                                                                                                                                                                                                                                          |
+-------------------+-------------------

In [40]:
# check the number of record in data_df dataframe
data_df.createOrReplaceTempView("df_data")
spark.sql("SELECT count(*) FROM df_data").show()

+--------+
|count(1)|
+--------+
|   47978|
+--------+



In [ ]:
spark.sql("SELECT count(DISTINCT id) FROM df_data").show()

In [ ]:
# flatten the neseted json to one record on row
place_df=flatten_nested_json(place_df)
place_df.show(500, False)

In [ ]:
# show the tweet location
place_df.show(500, False)

In [ ]:
# check the number of record in place_df dataframe
place_df.createOrReplaceTempView("df_data")
spark.sql("SELECT count(*) FROM df_data").show()

In [ ]:
# flatten the neseted json to one record on row
user_df=flatten_nested_json(user_df)

In [ ]:
# show the tweet user
user_df.show(100, False)

In [ ]:
# check the number of record in user_df dataframe
user_df.createOrReplaceTempView("df_data")
spark.sql("SELECT count(DISTINCT users_id) FROM df_data").show()

In [ ]:
spark.sql("SELECT users_name,users_location FROM df_data where users_location is not null").show(500,False)

In [41]:
from pyspark.ml.clustering import LDA
from pyspark.ml.feature import RegexTokenizer, StopWordsRemover, CountVectorizer
from gensim.models import LsiModel

In [42]:
# get the tweet text 
raw_tweets = data_df.select('id','text')

In [43]:
raw_tweets.show(10)

+-------------------+--------------------+
|                 id|                text|
+-------------------+--------------------+
|1384064534600507392|@hhjflrts im sorr...|
|1384064515759693830|@yogezlor By now ...|
|1384064394011709446|Forgot about the ...|
|1384064393017626635|Massive protester...|
|1384063860852678657|Just posted a pho...|
|1384063582938140673|@Ibraheema_Ykb Ok...|
|1384063508652851200|@mintea AND KIDS....|
|1384063029076127747|Just posted a pho...|
|1384062369492389892|@SheaSonia I am s...|
|1384062319781502977|In MyinGyan, terr...|
+-------------------+--------------------+
only showing top 10 rows



In [114]:
# Create a tokenizer that Filter away tokens with length < 4, and get rid of symbols like $,#,...
tokenizer = RegexTokenizer().setPattern("[\\W_]+").setMinTokenLength(4).setInputCol("text").setOutputCol("tokens")

In [115]:
#tokenizer = RegexTokenizer().setPattern("^[a-zA-Z]+\\b").setMinTokenLength(4).setInputCol("text").setOutputCol("tokens")

In [116]:
# Tokenize tweets
tokenized_tweets = tokenizer.transform(raw_tweets)

In [117]:
tokenized_tweets.select('text','tokens').show(50, False)

+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|text                                                                                                                                                                                                                                                                                                               |tokens                                                                                                          

In [72]:
# create cutomized extended stop word list
stopwordList = ["http","https","singapore","Singapore"]
StopWordsRemover().getStopWords()
stopwordList.extend(StopWordsRemover().getStopWords())
stopwordList = list(set(stopwordList))

In [73]:
# Create a remover that Filter away stop word
remover = StopWordsRemover(stopWords=stopwordList).setInputCol("tokens").setOutputCol("cleaned")

In [74]:
# remove stopwords
cleaned_tweets = remover.transform(tokenized_tweets)

In [75]:
cleaned_tweets.show()

+-------------------+--------------------+--------------------+--------------------+
|                 id|                text|              tokens|             cleaned|
+-------------------+--------------------+--------------------+--------------------+
|1384064534600507392|@hhjflrts im sorr...|[@hhjflrts, sorry...|[@hhjflrts, sorry...|
|1384064515759693830|@yogezlor By now ...|[@yogezlor, now, ...|[@yogezlor, know,...|
|1384064394011709446|Forgot about the ...|[forgot, about, t...|[forgot, wire, ta...|
|1384064393017626635|Massive protester...|[massive, protest...|[massive, protest...|
|1384063860852678657|Just posted a pho...|[just, posted, ph...|[posted, photo, f...|
|1384063582938140673|@Ibraheema_Ykb Ok...|[@ibraheema_ykb, ...|[@ibraheema_ykb, ...|
|1384063508652851200|@mintea AND KIDS....|[@mintea, and, ki...|[@mintea, kids., ...|
|1384063029076127747|Just posted a pho...|[just, posted, ph...|[posted, photo, h...|
|1384062369492389892|@SheaSonia I am s...|[@sheasonia, hear...|[@

In [76]:
# create a vector of words that at least appeared in two different tweets, and set maximum vocab size to 20000.
vectorizer = CountVectorizer().setInputCol("cleaned").setOutputCol("features").setVocabSize(20000).setMinDF(2).fit(cleaned_tweets)
wordVectors = vectorizer.transform(cleaned_tweets)#.select("id", "features")

In [77]:
wordVectors.show()

+-------------------+--------------------+--------------------+--------------------+--------------------+
|                 id|                text|              tokens|             cleaned|            features|
+-------------------+--------------------+--------------------+--------------------+--------------------+
|1384064534600507392|@hhjflrts im sorr...|[@hhjflrts, sorry...|[@hhjflrts, sorry...|(20000,[82,329,14...|
|1384064515759693830|@yogezlor By now ...|[@yogezlor, now, ...|[@yogezlor, know,...|(20000,[16,28,265...|
|1384064394011709446|Forgot about the ...|[forgot, about, t...|[forgot, wire, ta...|(20000,[347,685,7...|
|1384064393017626635|Massive protester...|[massive, protest...|[massive, protest...|(20000,[0,5,53,13...|
|1384063860852678657|Just posted a pho...|[just, posted, ph...|[posted, photo, f...|(20000,[1,2,52,25...|
|1384063582938140673|@Ibraheema_Ykb Ok...|[@ibraheema_ykb, ...|[@ibraheema_ykb, ...|(20000,[24,81,84,...|
|1384063508652851200|@mintea AND KIDS....|[@mi

In [78]:
# LDA
# create Latent Dirichlet Allocation model and run it on our data with 10 iteration and 10 topics
lda = LDA(k=10, maxIter=10)

In [79]:
# fit the model on data
ldaModel = lda.fit(wordVectors)

In [80]:
ll = ldaModel.logLikelihood(wordVectors)
lp = ldaModel.logPerplexity(wordVectors)

In [81]:
print("The lower bound on the log likelihood of the entire corpus: " + str(ll))
print("The upper bound on perplexity: " + str(lp))

The lower bound on the log likelihood of the entire corpus: -3396223.9018747434
The upper bound on perplexity: 9.381315678345791


In [82]:
# extract vocabulary from CountVectorizer
vocab = vectorizer.vocabulary

In [83]:
# create topics based on LDA
lda_topics = ldaModel.describeTopics()
lda_topics.show()

+-----+--------------------+--------------------+
|topic|         termIndices|         termWeights|
+-----+--------------------+--------------------+
|    0|[3, 290, 67, 451,...|[0.00260991212218...|
|    1|[222, 147, 17, 53...|[9.35711095740449...|
|    2|[133, 4, 17, 25, ...|[0.00270555428200...|
|    3|[3, 35, 44, 85, 7...|[0.01860420139377...|
|    4|[358, 213, 441, 2...|[0.00202599176617...|
|    5|[12, 11, 8, 13, 1...|[0.00592544553171...|
|    6|[691, 885, 1059, ...|[0.00173774502230...|
|    7|[1, 2, 66, 0, 151...|[0.03509606013838...|
|    8|[0, 5, 9, 19, 18,...|[0.02332235134214...|
|    9|[4, 32, 114, 28, ...|[0.00457563054971...|
+-----+--------------------+--------------------+



In [84]:
lda_topics.select('termWeights').show(10,False)

+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|termWeights                                                                                                                                                                                                                        |
+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|[0.0026099121221861525, 0.002525718500594252, 0.00241089814561297, 0.0018029014417863729, 0.0013291471935927134, 0.0012961057161425965, 0.0012457096686017024, 0.0010999240760467019, 0.001060076711215476, 0.0010522162537600795] |
|[9.357110957404492E-4, 7.496988884484846E-4, 6.410030606966744E-4, 6.2804942803

In [85]:
lda_topics.select('termIndices').show(10,False)

+-------------------------------------------------------+
|termIndices                                            |
+-------------------------------------------------------+
|[3, 290, 67, 451, 2, 1, 10, 6, 293, 461]               |
|[222, 147, 17, 538, 1415, 2015, 2158, 1866, 2172, 2746]|
|[133, 4, 17, 25, 148, 34, 24, 204, 272, 0]             |
|[3, 35, 44, 85, 74, 115, 79, 145, 139, 154]            |
|[358, 213, 441, 258, 499, 507, 607, 457, 723, 834]     |
|[12, 11, 8, 13, 10, 4, 20, 21, 15, 17]                 |
|[691, 885, 1059, 45, 222, 2031, 2023, 1860, 2044, 1504]|
|[1, 2, 66, 0, 151, 305, 98, 33, 459, 353]              |
|[0, 5, 9, 19, 18, 6, 45, 42, 7, 58]                    |
|[4, 32, 114, 28, 101, 23, 221, 53, 77, 367]            |
+-------------------------------------------------------+



In [86]:
topics_rdd = lda_topics.rdd
topics_words = topics_rdd\
       .map(lambda row: row['termIndices'])\
       .map(lambda idx_list: [vocab[idx] for idx in idx_list])\
       .collect()

for idx, topic in enumerate(topics_words):
    print("topic: {}".format(idx))
    print("*"*25)
    for word in topic:
       print(word)
    print("*"*25)

topic: 0
*************************
#singapore
bay
thanks
marina
photo
posted
get
&amp;
game
restaurant
*************************
topic: 1
*************************
call
nice
it’s
kill
writing
81352277
appointments.
strictly
sleepy
#88europets
*************************
topic: 2
*************************
fuck
like
it’s
i’m
that’s
don’t
still
pretty
baby
#whatshappeninginmyanmar
*************************
topic: 3
*************************
#singapore
#free
#zerowaste
@bts_twt
different
cool
city
smooth
ocean
tune
*************************
topic: 4
*************************
training
market
beauty
centre
ltd
pte
@sb19official
#bitcoin
#sb19
permanent
*************************
topic: 5
*************************
good
@barber_edward_
one
#edwardbarber
get
like
edward
barber
day
it’s
*************************
topic: 6
*************************
maid
hire
maid,
myanmar
call
#maid
#maidsinsingapore
portal
#housemaid
agency
*************************
topic: 7
*************************
posted
photo
vi